In [ ]:
from pyntcloud import PyntCloud
import os
import numpy as np
import random
from random import randint, uniform
import pandas as pd
import tensorflow as tf
import math
import time

from sklearn.mixture import GaussianMixture
from sklearn.mixture import BayesianGaussianMixture

#import open3d as o3d
import scipy.io as io
import scipy.ndimage as nd


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ User Settings ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Set size of Voxels. The higher the better but can also be very slow with big Pointclouds
voxel_size = 256

# Restore the trained model
if (voxel_size == 64):
    model = tf.keras.models.load_model('savedmodel/scannet_model_64.h5')
    print("CNN Model with Voxelsize of 64 has been loaded.")

if (voxel_size == 128):
    model = tf.keras.models.load_model('savedmodel/scannet_model_128.h5')
    print("CNN Model with Voxelsize of 128 has been loaded.")
    
if (voxel_size == 192):
    model = tf.keras.models.load_model('savedmodel/scannet_model_192.h5')
    print("CNN Model with Voxelsize of 192 has been loaded.")

if (voxel_size == 256):
    model = tf.keras.models.load_model('savedmodel/scannet_model_256.h5')
    print("CNN Model with Voxelsize of 256 has been loaded.")
          
# Clustering number of candidates 0-7
n_candidates = 1


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Functions ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Colorizes the different Clusters randomly
def colorize_pc(number_clusterpoints):

    rgb = np.zeros((number_clusterpoints, 3), dtype=int)

    r = randint(0, 254)
    g = randint(0, 254)
    b = randint(0, 254)

    rgb_temp = np.array((r,g,b))
    rgb = np.full_like(rgb,rgb_temp)

    return rgb

# Adding the Clusterlabel to the Clusters
def add_clusterlabel(number_clusterpoints, clusterlabel):
    
    clusterlabel_array = np.zeros((number_clusterpoints,1), dtype=int)
    clusterlabel_temp = np.array(clusterlabel)
    clusterlabel_array = np.full_like(clusterlabel_array, clusterlabel_temp)
    
    return clusterlabel_array
    

# Colorize the points and returns a dataframe with rgb and a dataframe with labels
def colorize_to_dataframe(clusters_dict, cnn_prediction):
    
    # Placeholder for the colorized Clusterscene (x,y,z,r,b,g(segment color), cluster_label)
    cluster_scene = np.zeros((0, 7), dtype=float)
    
    clusterlabel = 0
    for clusterlabel in range(cnn_prediction):
        cluster = clusters_dict[clusterlabel]
        number_clusterpoints = cluster.shape[0]
        colorize_rgb = colorize_pc(number_clusterpoints)
        
        # Adding Color to cluster
        cluster = np.hstack((cluster, colorize_rgb))

        #Adding Clusterlabel to Cluster
        cluster_label = add_clusterlabel(number_clusterpoints, clusterlabel)
        cluster = np.hstack((cluster, cluster_label))
        cluster_scene = np.append(cluster_scene, cluster, axis=0)
         
    x = cluster_scene[:, 0]
    y = cluster_scene[:, 1]
    z = cluster_scene[:, 2]
    # Segment rgb Color
    red = cluster_scene[:, 3]
    green = cluster_scene[:, 4]
    blue = cluster_scene[:, 5]
    # Cluster Label
    cluster_label = cluster_scene[:, 6]

    df_cluster_scene_rgb = pd.DataFrame(zip(x, y, z, red, green, blue), columns=["x", "y", "z", "red", "green", "blue"])
    df_cluster_scene_labels = pd.DataFrame(zip(x, y, z, cluster_label), columns=["x", "y", "z","cluster_label"])
    npy_cluster_scene_labels = df_cluster_scene_labels.to_numpy()
        
    return df_cluster_scene_rgb, npy_cluster_scene_labels


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


# load data from cvs    
def load_cvs_data(path):
    return np.loadtxt(path,delimiter=';')


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            # relative max occurence of label: arr[j][3][0]/arr[j][1]
            #if arr[j][3][0]/arr[j][1] < arr[j+1][3][0]/arr[j+1][1]:
            if arr[j][3].max() < arr[j+1][3].max():
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr

# compute an error per false clustered point
def compute_error(all_points, labels, orig_labels, orig_indices, orig_counts):
    assignments = {}
    # the total error
    error = 0
    # iterate over all clusters

    intervals = []

    label_values, label_counts = np.unique(labels, return_counts = True)
    orig_values, orig_label_counts = np.unique(all_points[:,3] , return_counts = True)

    #print(label_values, orig_values)
    #print(label_counts, orig_label_counts)

    diff = np.absolute(orig_label_counts.shape[0] - label_counts.shape[0])

    for i in range(orig_labels.shape[0]):
        idx = orig_indices[i]
        count = orig_counts[i]
        obj_labels = labels[idx : idx + count]
        #print(obj_labels)
        sorted_labels, obj_counts = np.unique(obj_labels, return_counts=True)
        intervals.append( (idx, count, sorted_labels, obj_counts) )
        #print(obj_counts.max())

    intervals = bubble_sort(intervals)
    n_unlabelled_points = 0
    for i in range(len(intervals)):
        obj_idx = intervals[i][0]
        len_points = intervals[i][1]
        sorted_labels = intervals[i][2]
        counts = intervals[i][3]
        #print(len_points, counts)

        # if label == -1 -> continue

        # the labels have the same shape as the point cloud hence we do not need to search over the points to know the original labels
        obj_labels = labels[obj_idx:obj_idx+len_points]

        n_unlabelled_points += len(np.argwhere(obj_labels == -1))

        #print(labels[last_len_points:last_len_points+len_points], new_labels, counts, len_points)
        #print(obj_labels, sorted_labels)
        # check if a label is already assigned
        is_already_assigned = True
        # check if a label is available
        no_label_available = False

        while is_already_assigned:
            # no more label available for assignment - this happens if there are less cluster than predicted
            if counts.size == 0:
                no_label_available = True
                break
            # get the index of the most frequent cluster
            j = np.argmax(counts)
            # get the most frequent cluster label
            if sorted_labels[j] == -1:
                is_already_assigned = True
            else:
                chosen_label = sorted_labels[j]
                # check if label is already assigned
                is_already_assigned = chosen_label in assignments.values()
            if(is_already_assigned):
                # if so delete the label and take the second most label and so forth
                sorted_labels = np.delete(sorted_labels, j)
                counts = np.delete(counts, j)
            else:
                break
        # if there are no more labels, consider all the points as misclustered
        # n_labels > n_orig_labels
        if no_label_available:
            false_points = np.argwhere(obj_labels != -1)
            error += len(false_points)
        else:
            # save the assignet label for next iterations
            assignments[i] = chosen_label
            #print(assignments)
            # filter the objec	ts with the wrong labels
            false_points = np.argwhere(obj_labels != chosen_label)
            false_points = np.argwhere(obj_labels[false_points] != -1)
            # increment the error for every false point
            error += len(false_points)
        #print(error)
    return error, assignments # n_unlabelled_points, diff, 

def compute_intervals(all_points):
    orig_labels, orig_indices, orig_counts = np.unique(all_points[:,3], return_index = True, return_counts = True)
    return orig_labels, orig_indices, orig_counts

def sort_point_cloud(all_points):
    all_points = all_points[all_points[:,3].argsort(kind='mergesort')]
    return all_points

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def remove_every_other(my_list):
    return my_list[::2]
    pass

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Start  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

"""
Scannet Data: 

1) vh_clean_2.labels.ply = Colorized Segments + Label

2) vh_clean_2.ply = Real RGB Colors - Labels but same Number of Points as 1)
"""

# Get RGB Pointcloud and Labels
DATADIR = "G:\datasets\scannet_ply_rgb\scans\\"
DATADIR_labels = "G:\datasets\scannet_ply_labels\scans\\"

folder = random.choice(os.listdir(DATADIR))
path = DATADIR + folder
file = next(os.walk(path))[2][0]
path = path +"\\"+ file

# Load Data to PyntCloud Object
pointcloud_rgb = PyntCloud.from_file(path)  #Format: x,y,z, r, g, b, alpha
initial_pointcloud = PyntCloud.from_file(path)
del pointcloud_rgb.points['alpha'] 

#~~~~~
path_labels = DATADIR_labels + folder + "\\"
file_labels = next(os.walk(path_labels))[2][0]
path_labels = path_labels + file_labels

# Load Data to PyntCloud Object
pointcloud_labels = PyntCloud.from_file(path_labels)  # Format: x,y,z, seg r, seg g, seg b,alpha,label

# Creating Pointcloud with rgb and labels
pointcloud_rgb.points['label'] = pointcloud_labels.points['label']

# Convert to npy for manipulation
npy_pointcloud = pointcloud_rgb.points.to_numpy()

# Sort the Pointcloud after labels
pointcloud_rgb = sort_point_cloud(npy_pointcloud)

# Get Number of Objects in the Scene
category = pointcloud_rgb[:, 6]
category = np.unique(category) # counts only each unique number
number_objects = category.shape[0]

#pointcloud_rgb = remove_every_other(pointcloud_rgb)

print("Number of Objects:", number_objects)

# Create pandas dateframe to convert x,y,z scene to pyntcloud Object
p_x = pointcloud_rgb[:, 0]
p_y = pointcloud_rgb[:, 1]
p_z = pointcloud_rgb[:, 2]

p_r = pointcloud_rgb[:, 3] 
p_g = pointcloud_rgb[:, 4] 
p_b = pointcloud_rgb[:, 5]
p_label = pointcloud_rgb[:, 6]

df_pointcloud = pd.DataFrame(zip(p_x, p_y, p_z, p_r, p_g, p_b, p_label ),
                             columns=['x', 'y', 'z','red','green','blue', 'label']) # In Pnytcloud it has to be 'red', 'green', 'blue',

df_pointcloud_label = pd.DataFrame(zip(p_x, p_y, p_z,p_label ),
                                   columns=['x', 'y', 'z','label'])

pointcloud_label = df_pointcloud_label.to_numpy()


pointcloud_rgb = PyntCloud(df_pointcloud)
pointcloud_rgb.plot()

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Prepare Data for input in CNN ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Voxelgrid only takes the XYZ Informations
voxelgrid_id = pointcloud_rgb.add_structure("voxelgrid", n_x = voxel_size, n_y = voxel_size, n_z = voxel_size)
voxelscene = pointcloud_rgb.structures[voxelgrid_id]

# Create binary array from Voxelscene
binary_voxelscene = voxelscene.get_feature_vector(mode="binary")

# Prepare data for Network input
# A 3D Cnn expects an 4D Tensor as Input
binary_voxelscene = np.expand_dims(binary_voxelscene, axis=0)

# CNN prediction
cnn_out = model.predict(binary_voxelscene)
cnn_prediction = np.argmax(cnn_out)

print("Predicted Numbers of Objects:", cnn_prediction)
print("real number of Objects:", number_objects)

if cnn_prediction != number_objects:
    cnn_prediction = number_objects
    print("False Number of Objects detected. Continuuing with real data for testing")


In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ CLUSTERING ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#Prepare Pointcloud for Clustering

cluster_pointcloud = pointcloud_rgb.xyz


# Setting up the Clustering Candidates
gaussian_clustering = GaussianMixture(n_components=cnn_prediction, covariance_type='full',tol=1e-4, max_iter=400)
bayesian_clustering = BayesianGaussianMixture(n_components=cnn_prediction, 
                                              covariance_type='full',tol=1e-4, max_iter=400)

gaussian_clustering2 = GaussianMixture(n_components=cnn_prediction, covariance_type='full',tol=1e-5, max_iter=500)
bayesian_clustering2 = BayesianGaussianMixture(n_components=cnn_prediction, 
                                               covariance_type='full',tol=1e-5, max_iter=500)

gaussian_clustering3 = GaussianMixture(n_components=cnn_prediction, covariance_type='full',tol=1e-6, max_iter=800)
bayesian_clustering3 = BayesianGaussianMixture(n_components=cnn_prediction, 
                                               covariance_type='full',tol=1e-6, max_iter=800)

gaussian_clustering4 = GaussianMixture(n_components=cnn_prediction, covariance_type='full',tol=1e-7, max_iter=1000)
bayesian_clustering4= BayesianGaussianMixture(n_components=cnn_prediction, 
                                              covariance_type='full',tol=1e-7, max_iter=1000)

gaussian_clustering5 = GaussianMixture(n_components=cnn_prediction, covariance_type='full',tol=1e-8, max_iter=1500)
bayesian_clustering5 = BayesianGaussianMixture(n_components=cnn_prediction, 
                                              covariance_type='full',tol=1e-8, max_iter=1500)

gaussian_clustering6 = GaussianMixture(n_components=cnn_prediction, covariance_type='full',tol=1e-9, max_iter=2000)
bayesian_clustering6 = BayesianGaussianMixture(n_components=cnn_prediction,
                                              covariance_type='full',tol=1e-9, max_iter=2000)

gaussian_clustering7 = GaussianMixture(n_components=cnn_prediction, covariance_type='diag',tol=1e-4, max_iter=500)
bayesian_clustering7 = BayesianGaussianMixture(n_components=cnn_prediction,
                                              covariance_type='diag',tol=1e-4, max_iter=500)

gaussian_clustering8 = GaussianMixture(n_components=cnn_prediction, covariance_type='diag',tol=1e-6, max_iter=800)
bayesian_clustering8 = BayesianGaussianMixture(n_components=cnn_prediction,
                                              covariance_type='diag',tol=1e-6, max_iter=800)

# List of Labels per Clustering
gm ={}
gm_labels={}
bm={}
bm_labels={}

gm_clusters_dict={}
gm_clusterscene_rgb={}
gm_clusterscene_labels={}

bm_clusters_dict={}
bm_clusterscene_rgb={}
bm_clusterscene_labels={}

gm_clusterscene_rgb_pynt={}
bm_clusterscene_rgb_pynt ={}


i = 0
j = 0

while i <= n_candidates:
    
    if i == 0:
        print("full 1")
        gm[i] = gaussian_clustering.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering.predict(cluster_pointcloud) 
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        
    if i == 1:
        print("full 2")
        gm[i] = gaussian_clustering2.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering2.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering2.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering2.predict(cluster_pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        
    if i == 2:
        print("full 3")
        gm[i] = gaussian_clustering3.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering3.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering3.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering3.predict(pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        
    if i == 3:
        print("full 4")
        gm[i] = gaussian_clustering4.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering4.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering4.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering4.predict(cluster_pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        
    if i == 4:
        print("full 5")        
        gm[i] = gaussian_clustering5.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering5.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering5.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering5.predict(cluster_pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        
    if i == 5:
        print("full 6")        
        gm[i] = gaussian_clustering6.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering6.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering6.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering6.predict(cluster_pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
    
    if i == 6:
        print("diag 1")        
        gm[i] = gaussian_clustering7.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering7.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering7.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering7.predict(cluster_pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
    
    if i == 5:
        print("diag 2")        
        gm[i] = gaussian_clustering8.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering8.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering8.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering8.predict(cluster_pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        
    # Colorize Clusters and adding Clusterlabels
    
    gm_clusterscene_rgb[i], gm_clusterscene_labels[i] = colorize_to_dataframe(gm_clusters_dict[i], cnn_prediction)
    bm_clusterscene_rgb[i], bm_clusterscene_labels[i] = colorize_to_dataframe(bm_clusters_dict[i], cnn_prediction)
    
    gm_clusterscene_rgb_pynt[i] = PyntCloud(gm_clusterscene_rgb[i])
    bm_clusterscene_rgb_pynt[i] = PyntCloud(gm_clusterscene_rgb[i])
    
    gm_clusterscene_rgb_pynt[i].plot(initial_point_size=0.1)
    bm_clusterscene_rgb_pynt[i].plot(initial_point_size=0.1)
    
    print("Cluster", i+1, "from", n_candidates+1, "done.")

    i=i+1
    

print("Segmentation done")

In [ ]:
# Check if Score Calc possible

initial_pointcloud = pointcloud_label

if gm_clusterscene_labels[0].shape != initial_pointcloud.shape:
    print("Score Calc not possible. Segmentated Pointclouds need to be the same size as Initial Pointcloud")
    print(gm_clusterscene_labels[0].shape)
    print(initial_pointcloud.shape)
    error_check = 0
else:
    run_count=1 # Only for testenvironment
    error_check = 1
    print("Score calulation possible")
    
    # ------------------------------------- Error calculation of Clusters --------------------------------------------------
if error_check == 1: 
    
    if run_count == 1: # Only for the test environment can be deleted in normal build

        # Ground Truth Pointcloud
        orig_labels, orig_indices, orig_counts = compute_intervals(initial_pointcloud)
        initial_n_points = initial_pointcloud.shape[0]

    run_count = 0


    #Calculate Score for each Cluster Methods Cluster
    gm_n_errornous_points = {}
    gm_assignments = {}
    gm_error = {}

    bm_n_errornous_points = {}
    bm_assignments = {}
    bm_error ={}

    i = 0
    while i <= n_candidates:

        if i == 0:

            #Ground Trouth Only for TESTING! should be 1
            gt_n_errornous_points, gt_assignments = compute_error(initial_pointcloud, initial_pointcloud[:,3],
                                                                  orig_labels, orig_indices, orig_counts)
            gt_error = 1 - ((gt_n_errornous_points)/initial_n_points)
            print("Error Ground Thruth", gt_error)

        gm_n_errornous_points[i], gm_assignments[i] = compute_error(initial_pointcloud, gm_clusterscene_labels[i][:,3],
                                                              orig_labels, orig_indices, orig_counts)

        bm_n_errornous_points[i], bm_assignments[i] = compute_error(initial_pointcloud, bm_clusterscene_labels[i][:,3],
                                                              orig_labels, orig_indices, orig_counts)

        # Statistics
        gm_error[i] = 1 - ((gm_n_errornous_points[i])/initial_n_points)
        bm_error[i] = 1 - ((bm_n_errornous_points[i])/initial_n_points)

        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        print("Error Gaussian Clustering with Setup:", gm[i], "lead to an Error of:", gm_error[i])
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        print("Error Bayesian Clustering with Setup:", bm[i], "lead to an Error of:", bm_error[i])
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

        i=i+1

    print("Error calculation done.")